In [1]:
import pandas as pd

In [2]:
# Read in student and school data
student_data_original = "Resources/students_complete.csv"
school_data_original = "Resources/schools_complete.csv"

# Build csvreader for each file
student_data_df = pd.read_csv(student_data_original)
# student_data_df.head()

# Second csvreader
school_data_df = pd.read_csv(school_data_original)
# school_data_df

# Alphabatize the school dataframe
sorted_school_data = school_data_df.sort_values("school_name")
# sorted_school_data

# Merge data sets based on school name
combined_data = pd.merge(student_data_df, sorted_school_data, how="left", on=["school_name", "school_name"])
# combined_data.head()

In [3]:
# Distric Summary Below

In [4]:
# Calculate total number of schools as length of list of unique schools. Remember that index starts at 0
total_schools = sorted_school_data["school_name"].unique()
# len(total_schools) 

# Calculate total number of students as length of list of unique student ids (names might be repeated)
student_total = combined_data["Student ID"].unique()
# len(student_total)

total_students = len(student_total)
# total_students

# Calculate the total budget
total_budget = sum(school_data_df["budget"])
# total_budget

# Describe the data to check more calculations along the way
# combined_data.describe()

# Calculate the average math and reading score
average_math_score = combined_data["math_score"].mean()
# print(average_math_score)

average_reading_score = combined_data["reading_score"].mean()
# print(average_reading_score)

# Calculate the precentage of students that pass math 
# Don't do a list, do .loc

passed_math_df = combined_data.loc[combined_data["math_score"] > 69, "Student ID"]
# print(passed_math_df.head())

passed_math_count = len(passed_math_df)
# print(passed_math_count)

passed_math_percent = (passed_math_count / total_students) * 100 
# passed_math_percent

# Calculate percentage of students that passed reading

passed_reading_df = combined_data.loc[combined_data["reading_score"] > 69, "Student ID"]
# passed_reading_df.head()

passed_reading_count = len(passed_reading_df)
# print(passed_reading_count)

passed_reading_percent = (passed_reading_count / total_students) * 100
# passed_reading_percent

# Calculate the percentage of students that pass BOTH math and reading
# Merge the dataframes and only include students with data in both
# Should be an inner merge

passed_reading_and_math_df = pd.merge(passed_math_df, passed_reading_df, on="Student ID")
# passed_reading_and_math_df.head()

# Calculate percentage of students who passed both
passed_both_count = len(passed_reading_and_math_df)
# print(passed_both_count)

passed_both_percent = (passed_both_count / total_students) * 100
# print(passed_both_percent)

In [5]:
# Create dataframe to hold all results, capitalization matters
District_Summary_df = pd.DataFrame({
    "Total Schools": [len(total_schools)], 
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "Percent Passing Math": [passed_math_percent],
    "Percent Passing Reading": [passed_reading_percent],
    "Percent Passing Overall":[passed_both_percent]
})
District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [6]:
# Clean formatting (optional)
# Add commas? Other?

In [7]:
# School Summary Below

In [8]:
# Sort data to be alphabetical by school name
sorted_data = combined_data.sort_values("school_name")
# sorted_data

# Make dataframe with school name, type, budget, and size
school_summary_df = sorted_school_data[["school_name", "type", "size", "budget"]]
# school_summary_df

school_summary_df["Per Student Budget"] = school_summary_df["budget"] / school_summary_df["size"]
# school_summary_df

In [9]:
# Calculate average reading and math scores

# Try with groupby
sorted_grouped_data = sorted_data.groupby("school_name")
# sorted_grouped_data.count()

# Create list of student counts
student_count_list = sorted_grouped_data["Student ID"].count()
# student_count_list

# Start with list of scores
reading_score_list = sorted_grouped_data["reading_score"].sum()
# reading_score_list

# Divide total scores in reading_score_list by student_count
average_reading_score_by_school = reading_score_list / student_count_list
# average_reading_score_by_school

# Calculate average math score
# Start with list of scores
math_score_list = sorted_grouped_data["math_score"].sum()
# math_score_list.unique()

# Divide total math scores by studen_count
average_math_score_by_school = math_score_list / student_count_list
# average_math_score_by_school

In [10]:
# THIS IS THE GOOD CELL

# Calculate the percent of students who passed math
# Count those who passed for each school
# If this works, it is thanks to stack overflow geniuses
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
passing_math_score = sorted_data.drop(sorted_data[sorted_data.math_score < 70].index)
# passing_math_score.head()

# Calculate the percent of students who passed reading
# Count those who passed for each school
# If this works, it is thanks to stack overflow geniuses
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
passing_reading_score = sorted_data.drop(sorted_data[sorted_data.reading_score < 70].index)
# passing_reading_score.head()

# Group passing score dataframes by school
grouped_math = passing_math_score.groupby("school_name")
# grouped_math.count()
grouped_reading = passing_reading_score.groupby("school_name")
# grouped_reading.count()

# Create list of student counts for math and reading passing
passed_math_count_by_school = grouped_math["Student ID"].count()
# passed_math_count_by_school

passed_reading_count_by_school = grouped_reading["Student ID"].count()
# passed_reading_count_by_school

# Divide passing counts by total student counts and multiply by 100
passed_reading_percent = (passed_reading_count_by_school / student_count_list) * 100
# passed_reading_percent

passed_math_percent = (passed_math_count_by_school / student_count_list) * 100
# passed_math_percent

In [11]:
# Calculate the number of students who passed both math and reading
# Drop non-passing math scores from passing_reading_score dataframe

passed_overall = passing_reading_score.drop(passing_reading_score[passing_reading_score.math_score < 70].index)
# passed_overall

grouped_pass_overall = passed_overall.groupby("school_name")
# grouped_pass_overall.count()

passed_overall_by_school = grouped_pass_overall["Student ID"].count()
# passed_overall_by_school

passed_overall_percent = (passed_overall_by_school / student_count_list) * 100
# passed_overall_percent
                      

In [12]:
# Start score dataframe to merge with summary later
reading_math_df = pd.DataFrame({
    "Average Reading Score": average_reading_score_by_school,
    "Average Math Score": average_math_score_by_school,
    "Percent Passing Reading": passed_reading_percent,
    "Percent Passing Math": passed_math_percent,
    "Percent Passing Overall": passed_overall_percent
})
reading_math_df

,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
school_name,,,,,
Bailey High School,81.033963,77.048432,81.933280,66.680064,54.642283
Cabrera High School,83.975780,83.061895,97.039828,94.133477,91.334769
Figueroa High School,81.158020,76.711767,80.739234,65.988471,53.204476
Ford High School,80.746258,77.102592,79.299014,68.309602,54.289887
Griffin High School,83.816757,83.351499,97.138965,93.392371,90.599455
Hernandez High School,80.934412,77.289752,80.862999,66.752967,53.527508
Holden High School,83.814988,83.803279,96.252927,92.505855,89.227166
Huang High School,81.182722,76.629414,81.316421,65.683922,53.513884
Johnson High School,80.966394,77.072464,81.222432,66.057551,53.539172


In [13]:
# Final School Summary

# Merge reading_math_df with school_summary_df for school_summmary_final
school_summary_final = pd.merge(school_summary_df, reading_math_df, how="left", on=["school_name", "school_name"])
school_summary_final

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,91.334769
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,53.204476
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,90.599455
5,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,53.527508
6,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,89.227166
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,53.513884
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,53.539172
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,90.540541


In [14]:
# Clean up formatting in school_summary_final

In [15]:
# Top Schools Summary

# Sort schools by overall passing percent
schools_sort_top = school_summary_final.sort_values("Percent Passing Overall", ascending=False)
# schools_sort_top

# Reset index to drop those below top five performers
reindex_schools = schools_sort_top.reset_index(drop=True)
# reindex_schools

# Drop schools below index 5 for top schools
top_schools = reindex_schools.drop(reindex_schools[reindex_schools.index > 4].index)
top_schools

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,91.334769
1,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.308869,93.272171,90.948012
2,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,90.599455
3,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,90.582567
4,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,90.540541


In [16]:
# Bottom Schools Summary

# Sort schools by overall passing percent
schools_sort_bottom = school_summary_final.sort_values("Percent Passing Overall", ascending=True)
# schools_sort

# Reset index to drop those below top five performers
reindex_schools = schools_sort_bottom.reset_index(drop=True)
# reindex_schools

# Drop schools below index 5 for top schools
bottom_schools = reindex_schools.drop(reindex_schools[reindex_schools.index > 4].index)
bottom_schools

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.220055,66.366592,52.988247
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,53.204476
2,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,53.513884
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,53.527508
4,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,53.539172


In [17]:
# Clean up formatting

In [18]:
# Math Scores by Grade Summary
# Ask Kelly about if there is a better way
# What type of conditional would let me do this all at once?

In [19]:
# Call student_data_df to verify needed info
student_data_df

# Separate each grade students
ninth_grade = student_data_df.drop(student_data_df[student_data_df.grade != "9th"].index)
ninth_grade

tenth_grade = student_data_df.drop(student_data_df[student_data_df.grade != "10th"].index)
tenth_grade

eleven_grade = student_data_df.drop(student_data_df[student_data_df.grade != "11th"].index)
eleven_grade

twelve_grade = student_data_df.drop(student_data_df[student_data_df.grade != "12th"].index)
twelve_grade

# Try with groupby
group_ninth = ninth_grade.groupby("school_name")
group_ninth.count()

group_tenth = tenth_grade.groupby("school_name")
group_tenth.count()

group_eleven = eleven_grade.groupby("school_name")
group_eleven.count()

group_twelve = twelve_grade.groupby("school_name")
group_twelve.count()

# Find grade size
ninth_class_size = group_ninth["Student ID"].count()
ninth_class_size

tenth_class_size = group_tenth["Student ID"].count()
tenth_class_size

eleven_class_size = group_eleven["Student ID"].count()
eleven_class_size

twelve_class_size = group_twelve["Student ID"].count()
twelve_class_size

# Hold math totals
ninth_math_total = group_ninth["math_score"].sum()
ninth_math_total

tenth_math_total = group_tenth["math_score"].sum()
tenth_math_total

eleven_math_total = group_eleven["math_score"].sum()
eleven_math_total

twelve_math_total = group_twelve["math_score"].sum()
twelve_math_total

# Average math score list for each grade
ninth_math_avg = ninth_math_total / ninth_class_size
ninth_math_avg

tenth_math_avg = tenth_math_total / tenth_class_size
tenth_math_avg

eleven_math_avg = eleven_math_total / eleven_class_size
eleven_math_avg

twelve_math_avg = twelve_math_total / twelve_class_size
twelve_math_avg

school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
dtype: float64

In [20]:
# Create dataframe to hold summary
math_score_summary = pd.DataFrame({
#     "School Name": total_schools,
    "9th": ninth_math_avg,
    "10th": tenth_math_avg,
    "11th": eleven_math_avg,
    "12th": twelve_math_avg
})
math_score_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [21]:
# Clean up formatting

In [22]:
# Reading Scores by Grade Summary

In [23]:
# Using Grade Groups and class sizes from the math scores, calculate reading averages by grade

# Hold reading totals
ninth_reading_total = group_ninth["reading_score"].sum()
ninth_reading_total

tenth_reading_total = group_tenth["reading_score"].sum()
tenth_reading_total

eleven_reading_total = group_eleven["reading_score"].sum()
eleven_reading_total

twelve_reading_total = group_twelve["reading_score"].sum()
twelve_reading_total

# Average reading score list for each grade
ninth_reading_avg = ninth_reading_total / ninth_class_size
ninth_reading_avg

tenth_reading_avg = tenth_reading_total / tenth_class_size
tenth_reading_avg

eleven_reading_avg = eleven_reading_total / eleven_class_size
eleven_reading_avg

twelve_reading_avg = twelve_reading_total / twelve_class_size
twelve_reading_avg

school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
dtype: float64

In [24]:
# Create dataframe to hold summary
reading_score_summary = pd.DataFrame({
#     "School Name": total_schools,
    "9th": ninth_reading_avg,
    "10th": tenth_reading_avg,
    "11th": eleven_reading_avg,
    "12th": twelve_reading_avg
})
reading_score_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [25]:
# Clean Formatting

In [26]:
# Scores by School Spending
# Take a look at the three dataframes to determine how best 
# to bin, break, and re-assemble the data
# This will likely carry over to scores by size and type, as well
# For type, drop one and the other from school_summary_final
# Make bins and group by bins and use .mean()

In [27]:
# Recall dataframes to use here 
school_summary_df

# Recall dataframes to use
reading_math_df

# Recall dataframes to use here
school_summary_final

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,91.334769
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,53.204476
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,90.599455
5,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,53.527508
6,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,89.227166
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,53.513884
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,53.539172
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,90.540541


In [33]:
# Make bins and name groups to apply to school_summary_final
bins = [0, 585, 630, 645, 680]

# Name the groups
spending_ranges_per_student = ["<$585", "$585-630", "$630-645", "$645-680"]

# Make a new dataframe and drop the columns you don't need (use .columns)

# Cut the data into the bins and label
school_summary_final["Spending Ranges Per Student"] = pd.cut(school_summary_final["Per Student Budget"], bins, labels=spending_ranges_per_student, include_lowest=True)
school_summary_final

# Groupby spending ranges
grouped_spending_ranges = school_summary_final.groupby("Spending Ranges Per Student")
grouped_spending_ranges.mean()

,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
Spending Ranges Per Student,,,,,,,,
<$585,1592.000000,9.246042e+05,581.000000,83.933814,83.455399,96.610877,93.460096,90.369459
$585-630,2291.750000,1.421222e+06,615.500000,83.155286,81.899826,92.718205,87.133538,81.418596
$630-645,2830.500000,1.809705e+06,639.500000,81.624473,78.518855,84.391793,73.484209,62.857656
$645-680,4104.333333,2.675768e+06,652.333333,81.027843,76.997210,81.133951,66.164813,53.526855


In [34]:
grouped_spending_ranges

In [35]:
dir(grouped_spending_ranges)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_examples_doc',
 '_agg_see_also_doc',
 '_aggregate',
 '_aggregate_frame',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_define_paths',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_fill',
 '_ge

In [ ]:
pd.DataFrame